In [1]:
import os
import shutil
import subprocess
from datetime import datetime
import warnings
from PIL import Image
import exifread

In [2]:
# MAC ONLY
def getMDLSDatInfo(fileName):
    cmd = "mdls"\
          " -name kMDItemContentCreationDate" \
          " -name kMDItemContentModificationDate" \
          " -name kMDItemFSContentChangeDate" \
          " -name kMDItemFSCreationDate" \
          " '{:s}'".format(fileName)
    ret = subprocess.run(cmd,capture_output=True, shell=True)
    # check for errors:
    err = ret.stderr.decode('utf-8').split("\n")
    if not err[0] == '':
        raise Exception(err)
    # parse "ret":
    pOut = ret.stdout.decode('utf-8').split("\n")
    output = {}
    # get dates:
    output['created'] = datetime.strptime(pOut[0].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    output['modified'] = datetime.strptime(pOut[1].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    output['file_changed'] = datetime.strptime(pOut[2].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    output['file_created'] = datetime.strptime(pOut[3].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    
    return output

In [3]:
def getExifTags(filename=''):
    if os.path.isfile(filename):
        f = open(filename, 'rb')
        tags = exifread.process_file(f)
        outTags = {}
        for tag in tags.keys():
            if tag not in ('JPEGThumbnail', 'TIFFThumbnail', 'Filename', 'EXIF MakerNote'):
#                 print("Key: {:s}, value {:s}".format(tag, tags[tag].printable))
                outTags[tag] = tags[tag]
        f.close()
        return tags
    else:
        return []

In [4]:
# set path to NAS scan:
root_path = "/Volumes/Multimedia/Archive/Photos/"

In [5]:
"""
What we want to do here, is recusively scan each directory, and for each file
that has extension .JPG .jpg .JPEG .jpeg, read the file name, and read the EXIF data
then compare the file name date & the 'EXIF DateTimeOriginal' tag to see the difference
"""

"\nWhat we want to do here, is recusively scan each directory, and for each file\nthat has extension .JPG .jpg .JPEG .jpeg, read the file name, and read the EXIF data\nthen compare the file name date & the 'EXIF DateTimeOriginal' tag to see the difference\n"

In [77]:
# list of acceptible extentions:
_exts = ['.jpg', '.jpeg']
# firstly, maybe just recusively generate all JPEG files..

file_list = []
for dirpath, subdirList, fileList in os.walk(root_path):
    for fname in fileList:
        _base, _ext = os.path.splitext(fname)
        if _ext.lower() in _exts:
            file_list.append(os.path.join(dirpath, fname))

In [79]:
def report(f):
    # get tag of jpeg file:
    _tags = getExifTags(f)
    # get file base name:
    base_name = f.split("/")[-1].split('.')[0]
    # take only first 24chars of base name...
    d_file = datetime.strptime(base_name[:24], '%Y_%m_%d-%H_%M_%S%z')
    if 'EXIF DateTimeOriginal' in _tags:
        # convert EXIF created date tag to date with 0 UTC
        _date = _tags['EXIF DateTimeOriginal'].printable + " +00:00"
        d_created = datetime.strptime(_date, '%Y:%m:%d %H:%M:%S %z')
       
        # get difference in timestamps:
        leader = 'exif' if d_created>d_file else 'file'
        dt = (d_create - d_file).seconds/3600 if leader=='exif' else (d_file - d_created).seconds/3600

        # REPORT [filename, EXIFDate, difference_hrs, leader]
        _wLine = f + ',' + d_file.strftime("%Y_%m_%d-%H_%M_%S%z") + ',' + d_created.strftime("%Y_%m_%d-%H_%M_%S%z") + ',' + str(dt) + ',' + leader
    else:
        print('File: {:s} does not have EXIF time data'.format(base_name))
        _wLine = f + ',' + d_file.strftime("%Y_%m_%d-%H_%M_%S%z") + ',null,null,null'

    return _wLine

In [78]:
len(file_list)

14049

In [80]:
lines = []
for f in file_list:
    lines.append(report(f))

File: 2020_01_19-19_23_48+0000 does not have EXIF time data
File: 2020_01_18-21_24_30+0000 does not have EXIF time data
File: 2020_01_19-19_27_05+0000 does not have EXIF time data
File: 2020_01_18-21_24_30+0000_001 does not have EXIF time data
File: 2019_09_09-03_41_36+0000 does not have EXIF time data
File: 2019_09_09-03_25_51+0000 does not have EXIF time data
File: 2019_09_09-03_33_28+0000_002 does not have EXIF time data
File: 2019_09_09-03_39_09+0000 does not have EXIF time data
File: 2019_09_04-01_37_12+0000 does not have EXIF time data
File: 2019_09_03-23_56_33+0000 does not have EXIF time data
File: 2019_09_09-02_32_05+0000 does not have EXIF time data
File: 2019_09_09-03_43_42+0000_001 does not have EXIF time data
File: 2019_09_12-00_20_25+0000 does not have EXIF time data
File: 2019_09_09-03_48_17+0000_001 does not have EXIF time data
File: 2019_09_09-03_19_32+0000 does not have EXIF time data
File: 2019_09_09-03_32_34+0000 does not have EXIF time data
File: 2019_09_09-03_25_2

Possibly corrupted field Tag 0x001A in MakerNote IFD


File: 2019_09_09-03_33_22+0000 does not have EXIF time data
File: 2019_09_09-03_25_26+0000 does not have EXIF time data
File: 2019_09_09-03_42_57+0000_003 does not have EXIF time data
File: 2019_09_09-03_49_14+0000_001 does not have EXIF time data
File: 2019_09_09-03_48_15+0000 does not have EXIF time data
File: 2019_09_09-03_43_30+0000 does not have EXIF time data
File: 2019_09_09-03_43_45+0000 does not have EXIF time data
File: 2019_09_09-03_33_54+0000_003 does not have EXIF time data
File: 2019_09_09-03_22_46+0000 does not have EXIF time data
File: 2019_09_09-03_39_00+0000_004 does not have EXIF time data
File: 2019_09_09-03_43_44+0000 does not have EXIF time data
File: 2019_09_09-03_43_13+0000 does not have EXIF time data
File: 2019_09_09-03_25_20+0000_003 does not have EXIF time data
File: 2019_09_09-03_48_08+0000_006 does not have EXIF time data
File: 2019_09_09-03_43_43+0000 does not have EXIF time data
File: 2019_09_09-03_43_35+0000 does not have EXIF time data
File: 2019_09_09

Possibly corrupted field Tag 0x001A in MakerNote IFD


File: 2019_09_09-03_43_58+0000 does not have EXIF time data
File: 2019_09_09-03_43_50+0000 does not have EXIF time data
File: 2019_09_09-03_40_56+0000 does not have EXIF time data
File: 2019_09_09-03_43_51+0000_001 does not have EXIF time data
File: 2019_09_09-03_43_06+0000_005 does not have EXIF time data
File: 2019_09_09-03_33_28+0000 does not have EXIF time data
File: 2019_09_09-03_48_08+0000_005 does not have EXIF time data
File: 2019_09_09-03_47_09+0000 does not have EXIF time data
File: 2019_09_09-03_38_54+0000_008 does not have EXIF time data
File: 2019_09_09-03_38_54+0000 does not have EXIF time data
File: 2019_09_09-03_49_06+0000_008 does not have EXIF time data
File: 2019_09_09-03_43_06+0000_003 does not have EXIF time data
File: 2019_09_09-03_33_47+0000_001 does not have EXIF time data
File: 2019_09_09-03_43_32+0000 does not have EXIF time data
File: 2019_09_09-03_42_57+0000 does not have EXIF time data
File: 2019_09_03-23_56_28+0000 does not have EXIF time data
File: 2019_0

...

File: 2011_09_18-21_44_17+0000 does not have EXIF time data
File: 2011_06_28-23_31_15+0000 does not have EXIF time data
File: 2011_12_30-17_05_25+0000 does not have EXIF time data
File: 2011_12_30-18_03_35+0000 does not have EXIF time data
File: 2011_03_18-23_31_38+0000 does not have EXIF time data
File: 2011_03_19-00_58_20+0000 does not have EXIF time data
File: 2011_03_19-01_00_31+0000 does not have EXIF time data
File: 2011_04_22-01_19_19+0000 does not have EXIF time data
File: 2011_05_12-17_46_36+0000 does not have EXIF time data
File: 2011_05_28-01_32_50+0000 does not have EXIF time data
File: 2011_06_01-15_04_35+0000 does not have EXIF time data
File: 2011_06_04-00_19_54+0000 does not have EXIF time data
File: 2011_06_04-16_53_13+0000 does not have EXIF time data
File: 2011_06_04-16_57_02+0000 does not have EXIF time data
File: 2011_06_18-18_56_47+0000 does not have EXIF time data
File: 2011_06_19-04_55_43+0000 does not have EXIF time data
File: 2011_06_25-23_18_29+0000 does not 

Possibly corrupted field Tag 0x0000 in EXIF IFD


File: 2010_06_20-03_14_14+0000 does not have EXIF time data
File: 2010_05_14-15_33_37+0000 does not have EXIF time data


Possibly corrupted field Tag 0x0000 in EXIF IFD


File: 2010_08_06-11_32_58+0000 does not have EXIF time data
File: 2010_12_08-20_33_23+0000 does not have EXIF time data
File: 2010_06_13-16_48_34+0000 does not have EXIF time data
File: 2010_03_17-21_20_37+0000 does not have EXIF time data
File: 2010_04_11-04_05_55+0000 does not have EXIF time data
File: 2010_04_26-16_00_44+0000 does not have EXIF time data
File: 2010_05_14-16_35_54+0000 does not have EXIF time data
File: 2010_05_14-16_37_35+0000 does not have EXIF time data
File: 2010_06_02-12_03_14+0000 does not have EXIF time data
File: 2010_06_20-02_55_57+0000 does not have EXIF time data
File: 2009_08_13-13_22_55+0000 does not have EXIF time data
File: 2009_08_31-15_22_52+0000 does not have EXIF time data
File: 2009_10_14-12_36_16+0000 does not have EXIF time data
File: 2009_11_14-17_38_24+0000 does not have EXIF time data
File: 2009_08_13-13_23_07+0000 does not have EXIF time data
File: 2009_08_13-13_23_04+0000 does not have EXIF time data
File: 2009_04_30-22_52_52+0000 does not 

In [81]:
len(lines)

In [92]:
import csv
with open('jpegNasReport.csv', mode='w') as NASFile:
    NAS_writer = csv.writer(NASFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for l in lines:
        items = l.split(',')
        year = items[1].split('_')[0]
        newitems = [items[0]] + [year] + items[1:]
        NAS_writer.writerow(newitems)

In [91]:
items = lines[0].split(',')
print(items)
newitems = [items[0]] + [items[1].split('_')[0]] + items[1:]
print(newitems)

['/Volumes/Multimedia/Archive/Photos/2020/2020_01_19-19_23_48+0000.jpg', '2020_01_19-19_23_48+0000', 'null', 'null', 'null']
['/Volumes/Multimedia/Archive/Photos/2020/2020_01_19-19_23_48+0000.jpg', '2020', '2020_01_19-19_23_48+0000', 'null', 'null', 'null']


In [51]:
# report template:
f = file_list[0]# get tag of jpeg file:
_tags = getExifTags(f)
# convert EXIF created date tag to date with 0 UTC
_date = _tags['EXIF DateTimeOriginal'].printable + " +00:00"
d_created = datetime.strptime(_date, '%Y:%m:%d %H:%M:%S %z')
# get file base name:
base_name = f.split("/")[-1].split('.')[0]
# take only first 24chars of base name...
d_file = datetime.strptime(base_name[:24], '%Y_%m_%d-%H_%M_%S%z')

# get difference in timestamps:
leader = 'exif' if d_created>d_file else 'file'
dt = (d_create - d_file).seconds/3600 if leader=='exif' else (d_file - d_created).seconds/3600

# REPORT [filename, EXIFDate, difference_hrs, leader]
_wLine = f + ',' + d_file.strftime("%Y_%m_%d-%H_%M_%S%z") + ',' + d_created.strftime("%Y_%m_%d-%H_%M_%S%z") + ',' + str(dt) + ',' + leader

In [52]:
print(_date)
print(file_list[0])
print(_wLine)

2009:03:24 20:39:46 +00:00
/Volumes/Multimedia/Archive/Photos/2009/2009_03_25-00_39_46+0000.JPG
/Volumes/Multimedia/Archive/Photos/2009/2009_03_25-00_39_46+0000.JPG,2009_03_25-00_39_46+0000,2009_03_24-20_39_46+0000,4.0,file


In [40]:
d_created.strftime("%Y_%m_%d-%H_%M_%S%z")

'2009_03_24-20_39_46+0000'

In [22]:
d_created

datetime.datetime(2009, 3, 24, 20, 39, 46, tzinfo=datetime.timezone.utc)

In [32]:
d_created>d_file

False